SVM implementation

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('star_classification.csv')
df["class"].unique()
df = df[0:20000]
x = df.drop('class', axis=1)
x = x.drop("obj_ID", axis=1)
x = x.drop("spec_obj_ID", axis=1)
x = x.drop("rerun_ID", axis=1) 
y = df['class']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

classes = {'GALAXY':0, 'QSO':1, 'STAR':2 }
print(y.unique())
x

['GALAXY' 'QSO' 'STAR']


,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,redshift,plate,MJD,fiber_ID
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,2,79,0.634794,5812,56354,171
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,5,119,0.779136,10445,58158,427
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,2,120,0.644195,4576,55592,299
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,3,214,0.932346,9149,58039,775
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,3,137,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,116.742115,42.269971,22.84368,22.29973,20.75661,19.95866,19.47037,1402,4,70,0.476464,3666,55185,973
19996,118.188167,44.085136,25.64453,22.74920,20.78390,19.89102,19.35816,1402,4,84,0.501545,6376,56269,695
19997,123.109994,58.186645,19.60161,19.41832,19.41733,19.13950,19.05412,4264,6,155,1.854104,1873,54437,316
19998,359.461825,28.348221,19.39042,17.85960,17.23222,16.95590,16.89005,4152,1,103,0.000079,2803,54368,396


In [4]:
def transformYtoNum(y):
    # Transforms String classes to integers
    y_new = np.zeros((len(y)))
    for i in range(len(y)):
        y_new[i] = classes.get(y[i])

    return y_new

In [5]:
# Converting pd frames to numpy arrays
y_train = y_train.to_numpy()
x_train = x_train.to_numpy()
y_test = y_test.to_numpy()
x_test = x_test.to_numpy()

y_train_transformed = transformYtoNum(y_train)
y_test_transformed = transformYtoNum(y_test)

# prevent OVERFLOW in calculations
scaler=StandardScaler()
x_train_scaler=scaler.fit_transform(x_train)
x_test_scaler=scaler.fit_transform(x_test)

<b>SVM_multiclass.fit()</b>
1. Call <b>fit_binary()</b> for each class. (Train a binary model for each class)<br>

<b>SVM_multiclass.fit_binary()</b>
1. Transform Y so that when the correct class is the one we are training the model for -> 1 else -1
2. Initialize W (Weight) array with zeros
3. Calculate <b>margin</b>: $$y_{[i]} \cdot (\mathbf{x}_i \cdot \mathbf{w} - b) \geq 1$$ <br>
4. if <b>margin</b> >= 1 : W -= η*(2*λ*w) <b>else</b>$$\mathbf{w} \gets \mathbf{w} - \eta \cdot \left( 2 \lambda \mathbf{w} - \mathbf{x}_i \cdot y_{[i]} \right)$$<br><br>

<b>SVM_multiclass.predict()</b><br>
1.Compute for each class score: $$\mathbf{x}_{\text{test}} \cdot \mathbf{w} - b$$<br>
2.Pick the class with the maximum score

In [16]:
class SVM_multiclass:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=20):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.models = []

    def fit(self, x_train, y_train):

        for c in np.unique(y_train):
            print("Training class:", int(c))
            y_train_class = np.where(y_train == c, 1, -1)
            self.models.append(self.fit_binary(x_train, y_train_class))

    def fit_binary(self, X, y):
        n_samples, n_features = X.shape
        y_ = np.where(y <= 0, -1, 1)
        w = np.zeros(n_features)
        b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, w) - b) >= 1
                if condition:
                    w -= self.lr * (2 * self.lambda_param * w)
                else:
                    w -= self.lr * (2 * self.lambda_param * w - np.dot(x_i, y_[idx]))
                    b -= self.lr * y_[idx]
        return [w,b]

    def predict(self, x_test):
        scores = []
        for w,b in self.models:
            scores.append(list(np.dot(x_test, w) - b))

        scores = np.array(scores)
        predictions = np.argmax(scores, axis=0)
        return predictions

In [17]:
svm = SVM_multiclass()
svm.fit(x_train_scaler, y_train_transformed)


Training class: 0
Training class: 1
Training class: 2


In [20]:
predictions = svm.predict(x_test_scaler)

correct = 0
for i in range(len(predictions)):
     if predictions[i] == y_test_transformed[i]:
          correct += 1

print (f"Accuracy: {correct/len(y_test) * 100: .2f}%")

Accuracy:  85.03%
